In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-08-21"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
106,2024-08-21,Eua Wnba,20:30,Atlanta Dream F,Phoenix Mercury F,2.10,1.74,163.5,1.85,1.93,1.5,1.94,1.61,67d2GQcE,0.476190,0.574713,0.540541,0.518135,0.050903,0.0,0.0,NaN,NaN,142.65,21.340019,0.149597,152.516,21.410416,0.140381,0.0,0.0,137.06,154.56,0.0,0.0,0.0,0.0,0.132583,0.02993,0.131462,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
107,2024-08-21,Mundo Copa Do Mundo Feminina,23:30,Nova Zelândia F,México F,1.97,1.81,136.5,1.89,1.89,1.5,1.82,1.67,AVLopBdE,0.507614,0.552486,0.529101,0.529101,0.060100,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.059861,0.00000,0.060783,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos
